In [1]:
!pip install konlpy
!pip install nltk
!pip install faiss-cpu
!pip install -U FlagEmbedding
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 11.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 78.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 36.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 41.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 42.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━

In [2]:
import re
import json
import numpy as np
import pandas as pd
import torch
import json
from collections import defaultdict
from tqdm import tqdm
import math
from six import iteritems
from six.moves import range
import heapq
import pickle
from collections.abc import Iterable
from collections import Counter
import scipy.sparse as sp
from scipy.sparse import csr_matrix

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import ISRIStemmer  # For Arabic stemming
from konlpy.tag import Okt  # For Korean text processing
import nltk
nltk.download('stopwords')

import faiss
from FlagEmbedding import FlagModel

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
E0000 00:00:1731087586.740099      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/

## Load and process test data

In [3]:
corpus_file_path = '/kaggle/input/dis-project-1-document-retrieval/corpus.json/corpus.json'

# Load the corpus JSON file
with open(corpus_file_path, 'r') as f:
    corpus = json.load(f)
    
docids = [doc["docid"] for doc in corpus]
docids[0]

'doc-en-9633'

In [4]:
# Load the test queries
test_path = '/kaggle/input/dis-project-1-document-retrieval/test.csv'
test_df = pd.read_csv(test_path)
test_df.head()

,id,query_id,query,lang
0,0,q-en-0,What organization proposed listing PFOA under ...,en
1,1,q-en-2,What type of coating do ZM1130 - ZM1132 have?,en
2,2,q-en-4,What year did Deutsche Bank sell its stake in ...,en
3,3,q-en-5,Who expressed exasperation when Raphael and Mo...,en
4,4,q-en-7,Who commissioned Amy Beach to compose a choral...,en


In [5]:
# Initialize stop words and stemmers for each language
stop_words = {
    'en': set(stopwords.words('english')),
    'fr': set(stopwords.words('french')),
    'de': set(stopwords.words('german')),
    'es': set(stopwords.words('spanish')),
    'it': set(stopwords.words('italian')),
    'ar': set(stopwords.words('arabic')),
    'ko': set()  # Korean stop words can be manually added if needed
}

stemmers = {
    'en': SnowballStemmer('english'),
    'fr': SnowballStemmer('french'),
    'de': SnowballStemmer('german'),
    'es': SnowballStemmer('spanish'),
    'it': SnowballStemmer('italian'),
    'ar': ISRIStemmer(),  # Arabic stemmer from NLTK
#     'ko': Okt()  # Use Korean Okt tokenizer for morphological analysis
}


def preprocess(text, lang):
    """Preprocesses a single text entry by removing URLs, punctuation, stop words, and applying stemming.
    
    Args:
        text (str): The text to preprocess.
        lang (str): The language code for the text.
    
    Returns:
        str: The preprocessed text.
    """
    stemmer = stemmers.get(lang)
    stop_words_lang = stop_words.get(lang, set())

    # Remove URLs and punctuation
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Tokenize and preprocess based on language
    words = text.split()
    if lang == 'ko':
        # Special handling for Korean
#         words = [word for word, tag in stemmer.pos(text) if word not in stop_words_lang]
        words = words
    else:
        # Apply stop word removal and stemming
        words = [stemmer.stem(word.lower()) if lang != 'ar' else stemmer.stem(word) 
                 for word in words if word.lower() not in stop_words_lang]
    
    return ' '.join(words)

In [6]:
preprocessed_queries = []
for index, document in test_df.iterrows():
    preprocessed_text = preprocess(document['query'], document['lang'])

    # Add preprocessed text to the corpus with docid and lang
    preprocessed_queries.append(preprocessed_text)

## BM 25 retrieval

In [7]:
PARAM_K1 = 1.5
PARAM_B = 0.75
EPSILON = 0.25

class FastBM25:
    """Implementation of Best Matching 25 ranking function."""
    
    def __init__(self, corpus=None, load_path=None):
        """
        Parameters
        ----------
        corpus : list of list of str
            Given corpus.
        load_path : str, optional
            Path to load precomputed data.
        """
        if load_path:
            self.load(load_path)
        elif corpus:
            self.corpus_size = len(corpus)
            self.avgdl = 0
            self.doc_freqs = []
            self.idf = {}
            self.doc_len = {}
            self._initialize(corpus)
            self.corpus = corpus
            self.get_score_by_reversed_index_all_documents(corpus)
        else:
            raise ValueError("Either corpus or load_path must be provided")

    def _initialize(self, corpus):
        """Calculates frequencies of terms in documents and in corpus. Also computes inverse document frequencies."""
        nd = {}  # word -> number of documents with word
        num_doc = 0
        for j, document in enumerate(tqdm(corpus, desc="Initializing corpus")):
            self.doc_len[j] = len(document)
            num_doc += len(document)

            frequencies = {}
            for word in document:
                frequencies[word] = frequencies.get(word, 0) + 1
            self.doc_freqs.append(frequencies)

            for word in frequencies:
                nd[word] = nd.get(word, 0) + 1

        self.avgdl = float(num_doc) / self.corpus_size

        idf_sum = 0
        negative_idfs = []
        self.nd = nd
        for word, freq in nd.items():
            idf = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)
            self.idf[word] = idf
            idf_sum += idf
            if idf < 0:
                negative_idfs.append(word)
        
        self.average_idf = float(idf_sum) / len(self.idf)
        eps = EPSILON * self.average_idf
        for word in negative_idfs:
            self.idf[word] = eps

    def get_score_by_reversed_index_all_documents(self, corpus):
        """
        Build reverted index for documents like {word:{index:grades}}
        """
        document_score = {}
        for index, document in enumerate(tqdm(corpus, desc="Building reversed index")):
            q_id = index
            doc_freqs = self.doc_freqs[index]
            for word in document:
                if word not in doc_freqs:
                    continue
                score = (self.idf[word] * doc_freqs[word] * (PARAM_K1 + 1)
                          / (doc_freqs[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * self.doc_len[index] / self.avgdl)))
                if word not in document_score:
                    document_score[word] = {q_id: round(score, 2)}
                else:
                    document_score[word].update({q_id: round(score, 2)})
        self.document_score = document_score

    def top_k_sentence(self, document, k=1):
        """
        document: Iterable, to be retrieved
        Returns
        -------
        float
            List of [(nearest sentence,index,score)].
        """
        assert isinstance(document, Iterable), 'document is not iterable'
        score_overall = {}
        for word in document:
            if word not in self.document_score:
                continue
            for key, value in self.document_score[word].items():
                score_overall[key] = score_overall.get(key, 0) + value
        k_keys_sorted = heapq.nlargest(k, score_overall, key=score_overall.get)
        return [(item, score_overall.get(item, None)) for item in k_keys_sorted]

    def save(self, path):
        """Save the BM25 model to a file."""
        data = {
            'corpus_size': self.corpus_size,
            'avgdl': self.avgdl,
            'doc_freqs': self.doc_freqs,
            'idf': self.idf,
            'doc_len': self.doc_len,
            'document_score': self.document_score
        }
        with open(path, 'wb') as file:
            pickle.dump(data, file)
        print(f"Model saved to {path}")

    def load(self, path):
        """Load the BM25 model from a file."""
        with open(path, 'rb') as file:
            data = pickle.load(file)
        self.corpus_size = data['corpus_size']
        self.avgdl = data['avgdl']
        self.doc_freqs = data['doc_freqs']
        self.idf = data['idf']
        self.doc_len = data['doc_len']
        self.document_score = data['document_score']
        print(f"Model loaded from {path}")

In [8]:
bm25_model = FastBM25(load_path='/kaggle/input/bm25-model/bm25_model.pkl')

Model loaded from /kaggle/input/bm25-model/bm25_model.pkl


In [9]:
%%time
topk_docs = []
for query in tqdm(preprocessed_queries):
    query = query.split()
    top_k_results = bm25_model.top_k_sentence(query, k=10)
    topk_docs.append([docids[result[0]] for result in top_k_results]) 

100%|██████████| 2000/2000 [01:40<00:00, 19.86it/s] 

CPU times: user 1min 40s, sys: 270 ms, total: 1min 41s
Wall time: 1min 40s


In [20]:
top_bm25 = pd.DataFrame({
    'id': [i for i in range(len(topk_docs))],
    'docids': topk_docs
})
top_bm25.head()

,id,docids
0,0,"[doc-en-0, doc-en-14117, doc-en-794977, doc-en..."
1,1,"[doc-en-16, doc-en-773190, doc-en-369832, doc-..."
2,2,"[doc-en-24920, doc-en-32, doc-en-420197, doc-e..."
3,3,"[doc-en-814022, doc-en-40, doc-en-535898, doc-..."
4,4,"[doc-en-701524, doc-en-56, doc-en-11186, doc-e..."


In [ ]:
top_bm25.to_csv('/kaggle/working/submission.csv', index=False)

In [11]:
del corpus
del bm25_model

## Dense retrieval

In [12]:
# %%time

# # Load the document embeddings
# with open('/kaggle/input/m3-embedding-of-512-chunks/chunk_embedding.pkl', 'rb') as f:
#     chunk_embedding_dict = pickle.load(f)
    
# chunk_ids = list(chunk_embedding_dict.keys())
# chunk_embeddings = np.array([chunk_embedding_dict[chunk_id] for chunk_id in chunk_ids]).astype('float32')

CPU times: user 12.5 s, sys: 4.05 s, total: 16.6 s
Wall time: 36.8 s


In [13]:
# # Load the model
# model = FlagModel('BAAI/bge-m3',
#                   query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
#                   use_fp16=True)

# # Embed the test queries
# queries = test_df['query'].tolist()
# query_ids = test_df['id'].tolist()
# query_embeddings = model.encode(queries).astype('float32')

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 8/8 [02:33<00:00, 19.14s/it] 


In [14]:
# %%time

# k = 100  # Number of nearest neighbors to retrieve

# # Normalize the corpus matrix
# faiss.normalize_L2(chunk_embeddings)

# # Initialize a FAISS index
# d = chunk_embeddings.shape[1]  # Dimensionality of embeddings
# index = faiss.IndexFlatIP(d)  # IP = Inner Product, effectively cosine similarity after normalization
# index.add(chunk_embeddings)  # Add document embeddings to the FAISS index

# # Normalize query embeddings
# faiss.normalize_L2(query_embeddings)

# # Perform the search and retrieve top 100 results
# distances, indices = index.search(query_embeddings, k)

CPU times: user 25min 31s, sys: 5.97 s, total: 25min 37s
Wall time: 1min 47s


In [15]:
# # Map the indices back to document IDs
# top_k_chunks = {
#     query_id: [chunk_ids[idx] for idx in indices[i]]
#     for i, query_id in enumerate(query_ids)
# }

# # Optional: Convert results to a DataFrame for easier access
# top_k_chunks_df = pd.DataFrame({
#     'id': query_ids,
#     'chunkids': [top_k_chunks[qid] for qid in query_ids]
# })

# # Display the top results
# top_k_chunks_df.head()

,id,chunkids
0,0,"[doc-en-0_chunk_13, doc-en-0_chunk_8, doc-en-1..."
1,1,"[doc-en-16_chunk_57, doc-en-16_chunk_56, doc-e..."
2,2,"[doc-it-14111_chunk_5, doc-en-32_chunk_7, doc-..."
3,3,"[doc-en-563220_chunk_1, doc-en-44779_chunk_1, ..."
4,4,"[doc-en-772504_chunk_0, doc-en-56_chunk_13, do..."


In [16]:
# def rank_and_aggregate(df, aggregate_num=20):
#     results = {}

#     # Loop through each query's candidates
#     for _, row in df.iterrows():
#         doc_scores = defaultdict(float)
#         query_id = row['id']
#         chunks = row['chunkids']

#         # Weight each chunk based on its position in the list (higher rank -> higher weight)
#         for rank, chunk_id in enumerate(chunks, start=1):
#             # Extract the document ID part (everything before "_chunk")
#             doc_id = "_".join(chunk_id.split("_")[:-2])
#             # Calculate weight, for example, inversely proportional to the rank
#             score = 1 / rank  # Adjust the weighting function if needed

#             # Aggregate scores for each document
#             doc_scores[doc_id] += score

#         # Get the top 10 documents based on cumulative scores
#         top_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)[:aggregate_num]
#         results[query_id] = [doc for doc, score in top_docs]

#     # Convert results to DataFrame for easier access
#     top_results_df = pd.DataFrame(list(results.items()), columns=['id', 'docids'])
#     return top_results_df

# # Apply the function
# top_dense = rank_and_aggregate(top_k_chunks_df)
# top_dense.head()  # Display the top results

,id,docids
0,0,"[doc-en-0, doc-en-14117, doc-de-14895, doc-en-..."
1,1,"[doc-en-16, doc-en-768850, doc-es-13327, doc-e..."
2,2,"[doc-en-32, doc-it-14111, doc-en-659327, doc-e..."
3,3,"[doc-en-563220, doc-en-751326, doc-en-44779, d..."
4,4,"[doc-en-772504, doc-en-56, doc-en-810421, doc-..."


## Weighted combine

In [37]:
# def aggregate_rankings(bm25_results, dense_results, top_k=10):
#     top_combined_results = []

#     # Assign scores based on rank positions for BM25 results
#     for i in range(len(bm25_results)):
#         combined_scores = defaultdict(float)
        
#         bm25_docids = bm25_results.loc[i, 'docids']
#         for rank, docid in enumerate(bm25_docids, start=1):
#             combined_scores[docid] += 0.45 * 1 / rank  # Higher rank gives higher score

#         dense_docids = dense_results.loc[i, 'docids']
#         for rank, docid in enumerate(dense_docids, start=1):
#             combined_scores[docid] += 0.55 * 1 / rank  # Higher rank gives higher score

#         # Sort by combined score and retrieve the top k
#         top_docs = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
#         top_docs = [item[0] for item in top_docs]
#         top_combined_results.append(top_docs)
#     return top_combined_results

# # Get the top 10 results
# top_10_combined_results = aggregate_rankings(top_bm25, top_dense, top_k=10)
# top_final = pd.DataFrame({
#     'id': query_ids,
#     'docids': top_10_combined_results
# })
# top_final

,id,docids
0,0,"[doc-en-0, doc-en-14117, doc-de-14895, doc-en-..."
1,1,"[doc-en-16, doc-en-768850, doc-en-773190, doc-..."
2,2,"[doc-en-32, doc-en-24920, doc-it-14111, doc-en..."
3,3,"[doc-en-814022, doc-en-563220, doc-en-751326, ..."
4,4,"[doc-en-772504, doc-en-56, doc-en-701524, doc-..."
...,...,...
1995,1995,"[doc-ar-8335, doc-en-1607, doc-ar-11184, doc-e..."
1996,1996,"[doc-ar-8351, doc-ar-12175, doc-en-17143, doc-..."
1997,1997,"[doc-ar-7659, doc-en-695959, doc-ar-2382, doc-..."
1998,1998,"[doc-ar-8394, doc-ar-59, doc-ar-5176, doc-en-7..."


In [43]:
# top_final.to_csv('/kaggle/working/submission.csv', index=False)